<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW2/blob/main/BigData_HW2_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark Setting

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 24 kB/s 
     |████████████████████████████████| 198 kB 45.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=17aed0a6c4c9a7445a59033ffbabdab7cc7bcd15220b3ff4ff2f594f780e568e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

# Libraries

In [ ]:
import pandas as pd
import numpy as np

In [219]:
from pyspark.sql.types import FloatType, ArrayType, IntegerType
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql import Window

# Dataset

In [ ]:
!gdown 1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
!unzip /content/Bigdata_hw2_datasets.zip

Downloading...
From: https://drive.google.com/uc?id=1PdXgb4w0gtsocKHmeGQ_zPfb3VbW9YhL
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 41.6MB/s]
Archive:  /content/Bigdata_hw2_datasets.zip
   creating: Bigdata_hw2_datasets/
   creating: Bigdata_hw2_datasets/q1/
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
   creating: Bigdata_hw2_datasets/q2/
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
   creating: Bigdata_hw2_datasets/q3/
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [ ]:
dataset_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/data.txt", header=None, names=["data"])
dataset_pandas["data"] = dataset_pandas["data"].apply(lambda x:x.split(" "))
dataset_pandas["data"] = dataset_pandas["data"].apply(lambda item: [float(x) for x in item])

c1_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/c1.txt", header=None, names=["center"])
c1_pandas["center"] = c1_pandas["center"].apply(lambda x:x.split(" "))
c1_pandas["center"] = c1_pandas["center"].apply(lambda item: [float(x) for x in item])

c2_pandas = pd.read_csv("/content/Bigdata_hw2_datasets/q3/c2.txt", header=None, names=["center"])
c2_pandas["center"] = c2_pandas["center"].apply(lambda x:x.split(" "))
c2_pandas["center"] = c2_pandas["center"].apply(lambda item: [float(x) for x in item])

In [ ]:
dataset_pandas["data-id"] = range(len(dataset_pandas))
c1_pandas["center-id"] = range(len(c1_pandas))

In [ ]:
dataset_spark = spark.createDataFrame(dataset_pandas)
center_spark = spark.createDataFrame(c1_pandas)

In [ ]:
NUMBER_FEATURES = 58

# KMeans

In [ ]:
def euclidean_distance(a, b):
  distance = 0
  for i in range(NUMBER_FEATURES):
    distance += (a[i] - b[i]) ** 2
  return distance ** 0.5

In [196]:
data_center_spark = dataset_spark.join(center_spark).withColumn('distance', euclidean_distance(dataset_spark.data, center_spark.center))

In [197]:
data_center_spark = data_center_spark.withColumn("min-distance", f.min("distance").over(Window.partitionBy("data-id")))\
                                    .where(f.col("min-distance") == f.col("distance"))\
                                    .drop(*["distance", "min-distance", "center"])

In [198]:
data_center_spark = data_center_spark.groupBy("center-id").agg(f.collect_list("data").alias("data-list"))

In [199]:
data_center_spark.show()

+---------+--------------------+
|center-id|           data-list|
+---------+--------------------+
|        0|[[0.0, 0.64, 0.64...|
|        7|[[0.0, 0.0, 0.0, ...|
|        6|[[0.0, 0.0, 0.0, ...|
|        9|[[0.06, 0.12, 0.7...|
|        5|[[0.0, 0.0, 0.0, ...|
|        1|[[0.21, 0.28, 0.5...|
|        3|[[0.0, 0.0, 0.0, ...|
|        8|[[0.15, 0.0, 0.46...|
|        2|[[0.06, 0.0, 0.71...|
|        4|[[0.0, 0.0, 0.0, ...|
+---------+--------------------+



In [223]:
data_center_spark.select(f.size(f.col("data-list"))).show()

+---------------+
|size(data-list)|
+---------------+
|            594|
|           1566|
|            708|
|            286|
|            597|
|            155|
|            226|
|            124|
|            105|
|            240|
+---------------+



In [240]:
def array_mean(x):
  number_data = len([i for i in zip(*x)][0])
  sum_array = [sum(i) for i in zip(*x)]
  return [item/number_data for item in sum_array]

array_mean_udf = udf(array_mean, ArrayType(FloatType()))

In [245]:
data_center_spark.select("center-id", array_mean_udf("data-list").alias("center"))

+---------+--------------------+
|center-id|              center|
+---------+--------------------+
|        0|[0.11368687, 0.13...|
|        7|[0.06644317, 0.38...|
|        6|[0.117189266, 0.1...|
|        9|[0.14741258, 0.15...|
|        5|[0.119798996, 0.0...|
|        1|[0.10509677, 0.13...|
|        3|[0.11137168, 0.16...|
|        8|[0.22782259, 0.25...|
|        2|[0.2707619, 0.182...|
|        4|[0.061166666, 0.1...|
+---------+--------------------+



In [226]:
# from pyspark.sql.functions import col, array, size, sum as sum_

# n = df.select(size("values")).first()[0]
# df = sc.parallelize([(1, [1, 2, 3]), (1, [4, 5, 6])]).toDF(["store", "values"])
# df.groupBy("store").agg(array(*[
#     sum_(col("values").getItem(i)) for i in range(n)]).alias("values"))

NameError: ignored